In [160]:
import pandas as pd
data=pd.read_csv("one_stock.csv")

In [161]:
data.set_index('Date',inplace=True)

In [162]:
l=int(len(data.index)*0.9)

In [163]:
data_train=data.iloc[:l]
data_test=data.iloc[l:]

In [164]:
train=data_train.to_numpy()

In [262]:
def getState(data, t, window=30):
    if t - window >= -1:
        vec = data.iloc[t - window + 1:t + 1,:].to_numpy()
    else: 
        x = np.tile(data.iloc[0,:].to_numpy(), (window-t-1, 1))
        y = data.iloc[0: t + 1,:].to_numpy()
        vec = np.vstack((x,y))
    return vec

In [476]:
class Simulator(object):

    def __init__(self, data, data_out,start_date, end_date):
        # frame a time period as world
        self.dates_range = pd.date_range(start_date, end_date)

        # initialize cash holdings
        init_cash = 100000

        #for visualization
        self.data_out = data_out
        
        
        # first trading day
        self.dateIdx = 0
        self.date = start_date
        self.start_date = start_date
        self.end_date = end_date
        
        self.price=data
        
        self.stock = data['Close']
        
        self.vol = data['Volume']
        self.ret = data['returns']
        self.close100 = data['close_pct_100']
        self.vol100 = data['volume_pct_100']
        self.close20 = data['close_pct_20']
        self.vol20 = data['volume_pct_20']
        self.ret5 = data['return_5']
        self.ret21 = data['return_21']
        self.rsi = data['rsi']
    

        # keep track of portfolio value as a series
        self.portfolio = {'cash': init_cash, 'vol': [], 'price': [],'long':0}
        self.port_val = 100000

        
    def init_state(self):
        """
        return init states of the market
        """
        self.dateIdx=0
        states=getState(self.price,self.dateIdx,30)
        #self.dateIdx += 1
        self.date = self.stock.index[self.dateIdx]
        return states

    def step(self, action):

        buy_volume = 100

        if (action == 'buy'):
            if (self.portfolio['long'] >= 0):
                long_cost = buy_volume * self.stock[self.dateIdx]
                self.portfolio['cash'] -= long_cost    
                self.portfolio['vol'].append(buy_volume)
                self.portfolio['price'].append(self.stock[self.dateIdx])
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                reward = self.port_val - old_port_val
            else:
                short_return = self.portfolio['vol'][0] * self.portfolio['price'][0]
                short_return += self.portfolio['vol'].pop(0) * (self.portfolio['price'].pop(0) -self.stock[self.dateIdx])         
                self.portfolio['cash'] += short_run
                self.portfolio['long'] = -1 if (len(self.portfolio['a_vol']) > 0) else 0
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                reward = self.port_val - old_port_val
                    
        elif (action == 'sell'):
            if (self.portfolio['long'] > 0):
                long_initial = self.portfolio['vol'][0] * self.portfolio['price'][0]
                long_return = self.portfolio['vol'].pop(0) * self.stock[self.dateIdx]
                self.portfolio['price'].pop(0)
                self.portfolio['cash'] += long_return-long_initial
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                reward = self.port_val - old_port_val
            else: # longA <= 0 --> buy in reverse
                short_cost = buy_volume * self.stock[self.dateIdx]
                self.portfolio['cash'] -= short_cost
                self.portfolio['vol'].append(buy_volume)
                self.portfolio['price'].append(self.stock[self.dateIdx])
                self.portfolio['long'] = -1
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                reward = self.port_val - old_port_val           
                    
        else: #hold 
            old_port_val = self.port_val
            self.port_val = self.port_value_for_output()
            reward = self.port_val - old_port_val
        
        self.data_out.append([self.port_val/100000-1,self.port_val])
        
        state =getState(self.price,self.dateIdx,30)
        
        self.dateIdx += 1
        done = False
        if self.dateIdx < len(self.stock.index)-1:
            self.date = self.stock.index[self.dateIdx]
        else:
            value = self.portfolio['cash']
            if (len(self.portfolio['vol']) > 0):
                for i in range(len(self.portfolio['vol'])):
                    value += (self.portfolio['vol'][i] * self.portfolio['price'][i])
            print("final step", value)
            done = True
        return (state,reward, done)
    
    # alternate calculation of the current value of cash and stock holdings
    def port_value_for_output(self):
        value = self.portfolio['cash']
        if (self.portfolio['long'] > 0):
            value += sum(self.portfolio['vol']) * self.stock[self.dateIdx]
        if (self.portfolio['long'] < 0):
            for i in range(len(self.portfolio['vol'])):
                value += (self.portfolio['vol'][i] * self.portfolio['price'][i])
                value += (self.portfolio['vol'][i] * (self.portfolio['price'][i] - self.stock[self.dateIdx]))
        return value


In [422]:
start_date=data_train.index[0]
end_date=data_train.index[-1]

In [448]:
data_train['Close'][-1]

79.67

In [449]:
data_train.tail()

,Close,Volume,returns,close_pct_100,volume_pct_100,close_pct_20,volume_pct_20,return_5,return_21,rsi
Date,,,,,,,,,,
2017-11-10,78.86,11.886556,0.002543,0.18,0.56,0.20,0.60,-1.834274,-1.289237,22.587968
2017-11-13,79.48,11.792071,0.007862,0.23,0.44,0.25,0.45,-1.803696,-2.874309,31.071429
2017-11-14,79.90,11.522876,0.005284,0.27,0.22,0.30,0.10,-1.173250,-0.171554,36.906212
2017-11-15,79.47,11.719940,-0.005382,0.23,0.40,0.25,0.40,-2.408398,-0.274161,27.237354
2017-11-16,79.67,11.552146,0.002517,0.26,0.25,0.35,0.15,-2.322260,-0.751329,28.535032


In [424]:
a=Simulator(data_train,data_out,start_date,end_date)
s,r,done=a.step('buy')
print(s.shape)

(30, 10)


In [458]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import time

In [459]:
LR_A = 0.001    # learning rate for actor
LR_C = 0.002    # learning rate for critic
GAMMA = 0.9     # reward discount
TAU = 0.01      # soft replacement
MEMORY_CAPACITY = 100
BATCH_SIZE = 30
TAU = 0.01

In [460]:
class Actor(nn.Module):   # ae(s)=a
    def __init__(self,s_dim,a_dim):
        super(Actor,self).__init__()
        
        self.fc1=torch.nn.LSTM(
            input_size=s_dim,
            hidden_size=20,
            dropout=0.2,
            num_layers=1)
        #self.fc1.weight.data.normal_(0,0.1) # initialization
        self.out = nn.Linear(20,a_dim)
        #self.out.weight.data.normal_(0,0.1) # initialization
    def forward(self,x):
        x= torch.unsqueeze(torch.FloatTensor(x), 1)
        #print(x.size())
        output,(h_n,c_n)=self.fc1(x)
        #print(output.size(),1)
        output_in_last_timestep=output[-1] # 也是可以的
        #print(output_in_last_timestep.size())
        s=torch.squeeze(output_in_last_timestep,1)
        #print(s.size(),2)
        x=self.out(s)
        actions_value = F.softmax(x,dim=-1)
        #print(actions_value.shape,3)
        return actions_value
        

In [407]:
from torch.autograd import Variable
a=Actor(10,3)
input_seq = Variable(torch.randn(30, 10))
last_output = a.forward(input_seq)

torch.Size([30, 1, 10])
torch.Size([30, 1, 20]) 1
torch.Size([1, 20])
torch.Size([1, 20]) 2
torch.Size([1, 3]) 3


In [461]:
class Critic(nn.Module):   # ae(s)=a
    def __init__(self,s_dim,a_dim):
        super(Critic,self).__init__()
        self.fcs=torch.nn.LSTM(
            input_size=s_dim,
            hidden_size=20,
            dropout=0.2,
            num_layers=1)
        #self.fcs.weight.data.normal_(0,0.1) # initialization
        self.fca = nn.Linear(a_dim,20)
        self.fca.weight.data.normal_(0,0.1) # initialization
        self.out = nn.Linear(20,1)
        self.out.weight.data.normal_(0, 0.1)  # initialization
    def forward(self,s,a):
        s= torch.unsqueeze(torch.FloatTensor(s), 1)
        #print(s.size())
        output,(h_n,c_n)=self.fcs(s)
        #print(output.size())
        output_in_last_timestep=output[-1] # 也是可以的
        #print(output_in_last_timestep.size())
        s=torch.squeeze(output_in_last_timestep,1)
        y = self.fca(a)
        net = F.relu(s+y)
        state_value = self.out(net)
        #print(state_value.shape)
        return state_value



In [439]:
class DDPG(object):
    def __init__(self, a_dim, s_dim):
        self.a_dim, self.s_dim= a_dim, s_dim
        self.memory = np.zeros((MEMORY_CAPACITY, s_dim * 2 + a_dim + 1), dtype=np.float32)
        self.pointer = 0
        #self.sess = tf.Session()
        self.Actor_eval = Actor(s_dim,a_dim)
        self.Actor_target = Actor(s_dim,a_dim)
        self.Critic_eval = Critic(s_dim,a_dim)
        self.Critic_target = Critic(s_dim,a_dim)
        self.ctrain = torch.optim.Adam(self.Critic_eval.parameters(),lr=LR_C)
        self.atrain = torch.optim.Adam(self.Actor_eval.parameters(),lr=LR_A)
        self.loss_td = nn.MSELoss()

    def choose_action(self, s):
        #print(self.Actor_eval(s).shape,4)
        return self.Actor_eval(s).detach().numpy()  # ae（s）

    def learn(self):

        for x in self.Actor_target.state_dict().keys():
            eval('self.Actor_target.' + x + '.data.mul_((1-TAU))')
            eval('self.Actor_target.' + x + '.data.add_(TAU*self.Actor_eval.' + x + '.data)')
        for x in self.Critic_target.state_dict().keys():
            eval('self.Critic_target.' + x + '.data.mul_((1-TAU))')
            eval('self.Critic_target.' + x + '.data.add_(TAU*self.Critic_eval.' + x + '.data)')

        # soft target replacement
        #self.sess.run(self.soft_replace)  # 用ae、ce更新at，ct

        indices = np.random.choice(MEMORY_CAPACITY, size=BATCH_SIZE)
        bt = self.memory[indices, :]
        bs = torch.FloatTensor(bt[:, :self.s_dim])
        ba = torch.FloatTensor(bt[:, self.s_dim: self.s_dim + self.a_dim])
        br = torch.FloatTensor(bt[:, -self.s_dim - 1: -self.s_dim])
        bs_ = torch.FloatTensor(bt[:, -self.s_dim:])

        a = self.Actor_eval(bs)
        q = self.Critic_eval(bs,a)  # loss=-q=-ce（s,ae（s））更新ae   ae（s）=a   ae（s_）=a_
        # 如果 a是一个正确的行为的话，那么它的Q应该更贴近0
        loss_a = -torch.mean(q) 
        #print(q)
        #print(loss_a)
        self.atrain.zero_grad()
        loss_a.backward()
        self.atrain.step()

        a_ = self.Actor_target(bs_)  # 这个网络不及时更新参数, 用于预测 Critic 的 Q_target 中的 action
        q_ = self.Critic_target(bs_,a_)  # 这个网络不及时更新参数, 用于给出 Actor 更新参数时的 Gradient ascent 强度
        q_target = br+GAMMA*q_  # q_target = 负的
        #print(q_target)
        q_v = self.Critic_eval(bs,ba)
        #print(q_v)
        td_error = self.loss_td(q_target,q_v)
        # td_error=R + GAMMA * ct（bs_,at(bs_)）-ce(s,ba) 更新ce ,但这个ae(s)是记忆中的ba，让ce得出的Q靠近Q_target,让评价更准确
        #print(td_error)
        self.ctrain.zero_grad()
        td_error.backward()
        self.ctrain.step()

    def store_transition(self, s, a, r, s_):
        #transition = np.hstack((s[-1], a, [r], s_[-1]))
        #print(s.size(),a,r)
        transition = np.zeros(self.s_dim * 2 + self.a_dim + 1)
        transition[:self.s_dim]=s[-1]
        transition[self.s_dim: self.s_dim + self.a_dim]=a
        transition[-self.s_dim - 1: -self.s_dim]=r
        transition[-self.s_dim:]=s_[-1]
        index = self.pointer % MEMORY_CAPACITY  # replace the old memory with new memory
        self.memory[index, :] = transition
        self.pointer += 1


In [387]:
from torch.autograd import Variable
a=Critic(10,3)
input_seq = Variable(torch.randn(30, 10))
ac_seq = Variable(torch.randn(1,3))
last_output = a.forward(input_seq,ac_seq)

In [440]:
agent=DDPG(3,10)
k=Simulator(data_train,data_out,start_date,end_date)
s=k.init_state()
print(s.shape)
a=agent.choose_action(s)
print(a.shape)
a = np.clip(np.random.normal(a, var), 0,2) # add randomness to action selection for exploration
a= np.argmax(a)
print(a)
action=actions[a]
s_, r, done = k.step(action)
print(r)

(30, 10)
(1, 3)
0
0.0


In [462]:
s_dim=10
a_dim=3
ddpg=DDPG(a_dim,s_dim)
var = 2  # control exploration
MAX_EPISODES = 1000
MAX_EP_STEPS = len(data_train.index)
data_out=[]
start_date=data_train.index[0]
end_date=data_train.index[1]
actions=['buy','sell','hold']

In [464]:
for i in range(MAX_EPISODES):
    Finance=Simulator(data_train,data_out,start_date,end_date)
    s = Finance.init_state()
    ep_reward = 0
    done = False
    #for j in range(MAX_EP_STEPS-1):
    while (done==False):
        # Add exploration noise
        a = ddpg.choose_action(s)
        a = np.clip(np.random.normal(a, var), 0,2) # add randomness to action selection for exploration
        a= np.argmax(a)
        action=actions[a]
        s_, r, done = Finance.step(action)

        ddpg.store_transition(s, a, r, s_)

        if ddpg.pointer > MEMORY_CAPACITY:
            var *= .9995    # decay the action randomness
            ddpg.learn()

        s = s_
        ep_reward += r
        if done == False:
            print('Episode:', i, ' Reward: %i' % int(ep_reward), 'Explore: %.2f' % var)


Episode: 0  Reward: 0 Explore: 2.00
Episode: 0  Reward: -2876 Explore: 2.00
Episode: 0  Reward: -2876 Explore: 2.00
Episode: 0  Reward: -2876 Explore: 2.00
Episode: 0  Reward: -2876 Explore: 2.00
Episode: 0  Reward: -2891 Explore: 2.00
Episode: 0  Reward: -5786 Explore: 2.00
Episode: 0  Reward: -5856 Explore: 2.00
Episode: 0  Reward: -5828 Explore: 2.00
Episode: 0  Reward: -8716 Explore: 2.00
Episode: 0  Reward: -11585 Explore: 2.00
Episode: 0  Reward: -14512 Explore: 2.00
Episode: 0  Reward: -14512 Explore: 2.00
Episode: 0  Reward: -14512 Explore: 2.00
Episode: 0  Reward: -14512 Explore: 2.00
Episode: 0  Reward: -14512 Explore: 2.00
Episode: 0  Reward: -17430 Explore: 2.00
Episode: 0  Reward: -17430 Explore: 2.00
Episode: 0  Reward: -17543 Explore: 2.00
Episode: 0  Reward: -17543 Explore: 2.00
Episode: 0  Reward: -17543 Explore: 2.00
Episode: 0  Reward: -17543 Explore: 2.00
Episode: 0  Reward: -17543 Explore: 2.00
Episode: 0  Reward: -20452 Explore: 2.00
Episode: 0  Reward: -20452 Exp

Episode: 0  Reward: -146117 Explore: 1.90
Episode: 0  Reward: -146406 Explore: 1.90
Episode: 0  Reward: -147398 Explore: 1.90
Episode: 0  Reward: -253623 Explore: 1.90
Episode: 0  Reward: -146348 Explore: 1.89
Episode: 0  Reward: -152052 Explore: 1.89
Episode: 0  Reward: -148819 Explore: 1.89
Episode: 0  Reward: -148027 Explore: 1.89
Episode: 0  Reward: -262801 Explore: 1.89
Episode: 0  Reward: -262801 Explore: 1.89
Episode: 0  Reward: -157743 Explore: 1.89
Episode: 0  Reward: -148153 Explore: 1.89
Episode: 0  Reward: -147154 Explore: 1.89
Episode: 0  Reward: -269058 Explore: 1.89
Episode: 0  Reward: -147724 Explore: 1.89
Episode: 0  Reward: -147373 Explore: 1.88
Episode: 0  Reward: -162545 Explore: 1.88
Episode: 0  Reward: -275256 Explore: 1.88
Episode: 0  Reward: -164761 Explore: 1.88
Episode: 0  Reward: -167808 Explore: 1.88
Episode: 0  Reward: -154985 Explore: 1.88
Episode: 0  Reward: -153351 Explore: 1.88
Episode: 0  Reward: -153546 Explore: 1.88
Episode: 0  Reward: -154266 Explor

Episode: 0  Reward: -487170 Explore: 1.72
Episode: 0  Reward: -487170 Explore: 1.72
Episode: 0  Reward: -176562 Explore: 1.72
Episode: 0  Reward: -174518 Explore: 1.72
Episode: 0  Reward: -292359 Explore: 1.71
Episode: 0  Reward: -292761 Explore: 1.71
Episode: 0  Reward: -493205 Explore: 1.71
Episode: 0  Reward: -299651 Explore: 1.71
Episode: 0  Reward: -171908 Explore: 1.71
Episode: 0  Reward: -496340 Explore: 1.71
Episode: 0  Reward: -496340 Explore: 1.71
Episode: 0  Reward: -496340 Explore: 1.71
Episode: 0  Reward: -167084 Explore: 1.71
Episode: 0  Reward: -500913 Explore: 1.71
Episode: 0  Reward: -500913 Explore: 1.71
Episode: 0  Reward: -307901 Explore: 1.71
Episode: 0  Reward: -175379 Explore: 1.70
Episode: 0  Reward: -503952 Explore: 1.70
Episode: 0  Reward: -503952 Explore: 1.70
Episode: 0  Reward: -303551 Explore: 1.70
Episode: 0  Reward: -502637 Explore: 1.70
Episode: 0  Reward: -502637 Explore: 1.70
Episode: 0  Reward: -170141 Explore: 1.70
Episode: 0  Reward: -507255 Explor

Episode: 0  Reward: -653154 Explore: 1.55
Episode: 0  Reward: -300570 Explore: 1.55
Episode: 0  Reward: -299913 Explore: 1.55
Episode: 0  Reward: -662957 Explore: 1.55
Episode: 0  Reward: -662957 Explore: 1.55
Episode: 0  Reward: -662957 Explore: 1.55
Episode: 0  Reward: -298137 Explore: 1.55
Episode: 0  Reward: -667887 Explore: 1.55
Episode: 0  Reward: -667887 Explore: 1.55
Episode: 0  Reward: -667887 Explore: 1.55
Episode: 0  Reward: -383520 Explore: 1.55
Episode: 0  Reward: -385100 Explore: 1.54
Episode: 0  Reward: -388647 Explore: 1.54
Episode: 0  Reward: -663353 Explore: 1.54
Episode: 0  Reward: -663353 Explore: 1.54
Episode: 0  Reward: -385616 Explore: 1.54
Episode: 0  Reward: -310330 Explore: 1.54
Episode: 0  Reward: -309322 Explore: 1.54
Episode: 0  Reward: -671840 Explore: 1.54
Episode: 0  Reward: -671840 Explore: 1.54
Episode: 0  Reward: -671840 Explore: 1.54
Episode: 0  Reward: -385916 Explore: 1.54
Episode: 0  Reward: -305708 Explore: 1.54
Episode: 0  Reward: -305854 Explor

Episode: 0  Reward: -483266 Explore: 1.41
Episode: 0  Reward: -484226 Explore: 1.41
Episode: 0  Reward: -965346 Explore: 1.40
Episode: 0  Reward: -965346 Explore: 1.40
Episode: 0  Reward: -493247 Explore: 1.40
Episode: 0  Reward: -970213 Explore: 1.40
Episode: 0  Reward: -970213 Explore: 1.40
Episode: 0  Reward: -489327 Explore: 1.40
Episode: 0  Reward: -975120 Explore: 1.40
Episode: 0  Reward: -482991 Explore: 1.40
Episode: 0  Reward: -980091 Explore: 1.40
Episode: 0  Reward: -510061 Explore: 1.40
Episode: 0  Reward: -503384 Explore: 1.40
Episode: 0  Reward: -979036 Explore: 1.40
Episode: 0  Reward: -508758 Explore: 1.40
Episode: 0  Reward: -978592 Explore: 1.40
Episode: 0  Reward: -978592 Explore: 1.39
Episode: 0  Reward: -978592 Explore: 1.39
Episode: 0  Reward: -978592 Explore: 1.39
Episode: 0  Reward: -515714 Explore: 1.39
Episode: 0  Reward: -978120 Explore: 1.39
Episode: 0  Reward: -978120 Explore: 1.39
Episode: 0  Reward: -524280 Explore: 1.39
Episode: 0  Reward: -527893 Explor

...

Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2732730 Explore: 0.44
Episode: 0  Reward: -2221155 Explore: 0.44
Episode: 0  Reward: -2729892 Explore: 0.44
Episode: 0  Reward: -2729892 Explore: 0.44
Episode: 0  Reward: -2729892 Explore: 0.44
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0  Reward: -2729892 Explore: 0.43
Episode: 0 

Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2778086 Explore: 0.40
Episode: 0  Reward: -2232469 Explore: 0.40
Episode: 0  Reward: -2231975 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.40
Episode: 0  Reward: -2772565 Explore: 0.39
Episode: 0  Reward: -2772565 Explore: 0.39
Episode: 0  Reward: -2772565 Explore: 0.39
Episode: 0  Reward: -1801029 Explore: 0.39
Episode: 0  Reward: -2778489 Explore: 0.39
Episode: 0  Reward: -2778489 Explore: 0.39
Episode: 0 

Episode: 0  Reward: -2799963 Explore: 0.36
Episode: 0  Reward: -2799963 Explore: 0.36
Episode: 0  Reward: -1588163 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0  Reward: -2807263 Explore: 0.36
Episode: 0 

Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.33
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0  Reward: -2854487 Explore: 0.32
Episode: 0 

KeyError: 'a_vol'

In [453]:
len(data_out)

22586

In [465]:
data_out

[[0.0, 100000.0],
 [-0.028760000000000008, 97124.0],
 [-0.028760000000000008, 97124.0],
 [-0.028760000000000008, 97124.0],
 [-0.028760000000000008, 97124.0],
 [-0.02890999999999999, 97109.0],
 [-0.05786000000000002, 94214.0],
 [-0.058559999999999945, 94144.0],
 [-0.05828, 94172.0],
 [-0.08716000000000002, 91284.0],
 [-0.11585000000000001, 88415.0],
 [-0.14512000000000003, 85488.0],
 [-0.14512000000000003, 85488.0],
 [-0.14512000000000003, 85488.0],
 [-0.14512000000000003, 85488.0],
 [-0.14512000000000003, 85488.0],
 [-0.1743, 82570.0],
 [-0.1743, 82570.0],
 [-0.17542999999999997, 82457.0],
 [-0.17542999999999997, 82457.0],
 [-0.17542999999999997, 82457.0],
 [-0.17542999999999997, 82457.0],
 [-0.17542999999999997, 82457.0],
 [-0.20452000000000004, 79548.0],
 [-0.20452000000000004, 79548.0],
 [-0.20452000000000004, 79548.0],
 [-0.20452000000000004, 79548.0],
 [-0.20452000000000004, 79548.0],
 [-0.20452000000000004, 79548.0],
 [-0.23204000000000002, 76796.0],
 [-0.20594, 79406.0],
 [-0.20

In [470]:
a=np.ones((10,3))

In [471]:
a.shape

(10, 3)

In [472]:
a=np.expand_dims(a,1)

In [473]:
a.shape

(10, 1, 3)

In [2]:
import pandas as pd
import yfinance as yf
data1= yf.download('STI','2002-01-01','2019-09-15')
data2= yf.download('HBAN','2002-01-01', '2019-09-15')
data3= yf.download('^GSPC','2002-01-01', '2019-09-15')
data4= yf.download('^VIX','2002-01-01', '2019-09-15')
data5= yf.download('^IXIC','2002-01-01', '2019-09-15')
data=pd.DataFrame({'STI':data1['Close'],'HBAN':data2['Close'],'SP500':data3['Close'],'VIX':data4['Close'],'DOW':data5['Close']})

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


In [3]:
data['spread']=data['STI']-data['HBAN']

In [4]:
data.to_csv('final_pair.csv')

In [6]:
data.isnull().values.any()

False

In [ ]:
data